In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LeakyReLU
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

from time import time

# CONSTANTS
IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128
IMAGE_NUMBER_CHANNELS = 3
NUM_CLASSES = 102

Using TensorFlow backend.


In [2]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_valid = np.load('x_valid.npy')
y_valid = np.load('y_valid.npy')

#Normalize data
x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_train = x_train / 255
x_valid = x_valid / 255

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, NUM_CLASSES)
y_valid = to_categorical(y_valid, NUM_CLASSES)

#MNIST resolution
img_rows, img_cols, channels = IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_NUMBER_CHANNELS

from tensorflow.keras import backend as K

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], channels, img_rows, img_cols)
    x_valid = x_valid.reshape(x_valid.shape[0], channels, img_rows, img_cols)
    #x_test = x_test.reshape(x_test.shape[0], channels, img_rows, img_cols)
    input_shape = (channels, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels)
    x_valid = x_valid.reshape(x_valid.shape[0], img_rows, img_cols, channels)
    #x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels)
    input_shape = (img_rows, img_cols, channels)

In [3]:
'''# Data load and splitting
X = np.load("X.npy")
Y = np.load("Y.npy")

Y = np_utils.to_categorical(Y, NUM_CLASSES)

mask = np.random.rand(len(X)) < 0.9

X_trn, X_tst = X[mask], X[~mask]
Y_trn, Y_tst = Y[mask], Y[~mask]'''

# Adaptacion
X_trn, X_tst = x_train, x_valid
Y_trn, Y_tst = y_train, y_valid

In [4]:
#CIFAR-10 ORIGINAL
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_trn.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

'''
# Prueba internet acc 60-70?
model = Sequential()

model.add(Conv2D(32, (5, 5), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(96, (5, 5)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#     model.add(Flatten())
model.add(GlobalAveragePooling2D())

model.add(Dense(512, activation='tanh'))
model.add(Dropout(0.25)) 

#    model.add(Dense(256, activation='tanh'))
#    model.add(Dropout(0.25)) 

model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.25)) 

model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))'''

# Let's compile the model using RMSprop
opt = tf.keras.optimizers.Adam(decay=0.001)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

# Comment if no TPU available
#model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=tf.contrib.tpu.TPUDistributionStrategy(
#    tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://10.240.1.2:8470'))
#)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)    

INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) fo

KeyboardInterrupt: 

In [ ]:
'''callbacks = [EarlyStopping(monitor='val_acc', patience=6)]

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_trn)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(
    datagen.flow(X_trn, Y_trn, batch_size=64),
    epochs=150,
    validation_data=(X_tst, Y_tst),
    workers=4,
    callbacks=callbacks)

data_generator = ImageDataGenerator(shear_range=0.3,
                                    zoom_range=0.3,
                                    rotation_range=30,
                                    horizontal_flip=True)

model.fit_generator(data_generator.flow(X_trn, Y_trn, batch_size=64),
                                                epochs=150, verbose=2, 
                                               validation_data=(X_tst, Y_tst))'''

In [ ]:
callbacks_list = [
    TensorBoard(log_dir="logs/{}".format(time())),
    EarlyStopping(monitor="val_acc", patience=10)
]

import time
start_time = time.time()

model.fit(X_trn, Y_trn,
              batch_size=64,
              epochs=999,
              validation_data=(X_tst, Y_tst),
              shuffle=True, callbacks=callbacks_list)

print("--- %s seconds ---" % (time.time() - start_time))